In [18]:
# !pip install -q llama-index-core
# !pip install -q llama-index-embeddings-mistralai
# !pip install -q llama-index-llms-mistralai
# !pip install -q llama-index-readers-file
# !pip install mistralai pypdf
# !pip install wget

In [1]:

import nest_asyncio

nest_asyncio.apply()

import os

In [2]:
from llama_index.llms.mistralai import MistralAI
from llama_index.embeddings.mistralai import MistralAIEmbedding
from llama_index.core import Settings

In [5]:
# from google.colab import userdata

# os.environ["MISTRAL_API_KEY"] = userdata.get('MISTRAL_API_KEY')
os.environ["MISTRAL_API_KEY"] = 'jLyhjBZgq9Epl3vAoPF1x4BjVsazFB3H'

In [6]:
llm = MistralAI(model="open-mixtral-8x22b", temperature=0.1)
embed_model = MistralAIEmbedding(model_name="mistral-embed")

Settings.llm = llm
Settings.embed_model = embed_model

### Download Data
We will use Uber-2021 and Lyft-2021 10K SEC filings.

In [10]:
import wget
!wget -q -O './uber_2021.pdf' https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf
!wget -q -O './lyft_2021.pdf' https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf

'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.


In [14]:
!curl -O https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf './uber_2021.pdf'
!curl -O https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf -O './lyft_2021.pdf'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1836k  100 1836k    0     0  3788k      0 --:--:-- --:--:-- --:--:-- 3858k
curl: (3) URL rejected: Bad hostname
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1406k  100 1406k    0     0  1669k      0 --:--:-- --:--:-- --:--:-- 1684k
curl: Failed to extract a sensible file name from the URL to use for storage
curl: (3) URL using bad/illegal format or missing URL


### Load the Data


In [15]:
from llama_index.core import SimpleDirectoryReader

uber_docs = SimpleDirectoryReader(input_files=["./uber_2021.pdf"]).load_data()
lyft_docs = SimpleDirectoryReader(input_files=["./lyft_2021.pdf"]).load_data()

In [16]:
# uber_docs

### Build RAG on uber and lyft docs

Separte Vector Stores for both companies


In [17]:
from llama_index.core import VectorStoreIndex

uber_index = VectorStoreIndex.from_documents(uber_docs)
uber_query_engine = uber_index.as_query_engine(similarity_top_k=5)

lyft_index = VectorStoreIndex.from_documents(lyft_docs)
lyft_query_engine = lyft_index.as_query_engine(similarity_top_k=5)

#### Ask questions

In [18]:
response = uber_query_engine.query("What is the revenue of uber in 2021?")
print(response)

Uber's revenue in 2021 was $17,455 million.


In [ ]:
# response

In [19]:
response = lyft_query_engine.query("What are lyft investments in 2021?")
print(response)

In 2021, Lyft invested in expanding their network of Light Vehicles and Lyft Autonomous, focusing on deploying and scaling third-party self-driving technology on their network. They also invested in their proprietary technology, such as mapping, routing, payments, in-app navigation, matching technologies, and data science, to deliver a convenient and high-quality experience to drivers and riders. Additionally, they invested in their Driver Centers, Mobile Services, and related partnerships to offer drivers affordable and convenient vehicle maintenance services. They also acquired Flexdrive, LLC, one of their longstanding Express Drive partners.


#### FunctionCallingAgent with RAG QueryEngineTools

Here we use Fuction Calling capabilities of the model. Automatically routing the query to appropriate VectorStore



In [20]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import FunctionCallingAgentWorker, AgentRunner

In [21]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_query_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description="Provides information about Lyft financials for year 2021",
        ),
    ),
    QueryEngineTool(
        query_engine=uber_query_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description="Provides information about Uber financials for year 2021",
        ),
    ),
]


In [22]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    allow_parallel_tool_calls=False,
)

In [23]:
agent = AgentRunner(agent_worker)

In [24]:
response = agent.chat("What is the revenue of uber in 2021.")

Added user message to memory: What is the revenue of uber in 2021.
=== Calling Function ===
Calling function: uber_10k with args: {"input": "revenue"}
=== Function Output ===
Uber's revenue is primarily derived from fees paid by Mobility Drivers for using their platforms and related services to facilitate and complete Mobility services. Additionally, revenue is generated from fees paid by end-users for connection services obtained via the platform in certain markets. Uber's revenue also includes immaterial revenue streams such as financial partnerships products and Vehicle Solutions.

Uber's Delivery revenue is derived from Merchants' and Couriers' use of the Delivery platform and related services to facilitate and complete Delivery transactions. In certain markets where Uber is responsible for delivery services, delivery fees charged to end-users are also included in revenue. Delivery revenue also includes advertising revenue from sponsored listing fees paid by merchants and brands in

In [25]:
print(response)

Uber's total revenue for the year 2021 was $17,455 million. This revenue is derived from various sources such as Mobility, Delivery, Freight, and All Other revenue streams. The Mobility revenue was $6,953 million, Delivery revenue was $8,362 million, Freight revenue was $2,132 million, and All Other revenue was $8 million.


In [26]:
response = agent.chat("What are lyft investments in 2021?")

Added user message to memory: What are lyft investments in 2021?
=== Calling Function ===
Calling function: lyft_10k with args: {"input": "investments"}
=== Function Output ===
The company's investments include debt securities such as cash and cash equivalents, short-term investments, and restricted investments. Cash equivalents include certificates of deposits, commercial paper, and corporate bonds with an original maturity of 90 days or less. Short-term investments are comprised of commercial paper, certificates of deposit, and corporate bonds that mature in twelve months or less. Restricted investments are debt security investments in commercial paper, certificates of deposit, corporate bonds, and U.S. government securities which are held in trust accounts at third-party financial institutions pursuant to certain contracts with insurance providers. The company also has investments in non-marketable equity securities, which are measured at cost with remeasurements to fair value only 

In [27]:
print(response)

Lyft's investments in 2021 primarily include debt securities such as cash and cash equivalents, short-term investments, and restricted investments. Cash equivalents include certificates of deposits, commercial paper, and corporate bonds with an original maturity of 90 days or less. Short-term investments are comprised of commercial paper, certificates of deposit, and corporate bonds that mature in twelve months or less. Restricted investments are debt security investments in commercial paper, certificates of deposit, corporate bonds, and U.S. government securities which are held in trust accounts at third-party financial institutions pursuant to certain contracts with insurance providers. The company also has investments in non-marketable equity securities, which are measured at cost with remeasurements to fair value only upon the occurrence of observable transactions for identical or similar investments of the same issuer or impairment.


### Agents and Tools usage

In [28]:
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import (
    FunctionCallingAgentWorker,
    AgentRunner,
    ReActAgent,
)

In [29]:
def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b


def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


def subtract(a: int, b: int) -> int:
    """Subtract two integers and returns the result integer"""
    return a - b


multiply_tool = FunctionTool.from_defaults(fn=multiply)
add_tool = FunctionTool.from_defaults(fn=add)
subtract_tool = FunctionTool.from_defaults(fn=subtract)

#### With Function Calling.

In [30]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [multiply_tool, add_tool, subtract_tool],
    llm=llm,
    verbose=True,
    allow_parallel_tool_calls=False,
)
agent = AgentRunner(agent_worker)

In [31]:
response = agent.chat("What is (26 * 2) + 2024?")
print(response)

Added user message to memory: What is (26 * 2) + 2024?
=== Calling Function ===
Calling function: multiply with args: {"a": 26, "b": 2}
=== Function Output ===
52
=== Calling Function ===
Calling function: add with args: {"a": 52, "b": 2024}
=== Function Output ===
2076
=== LLM Response ===
The result of (26 * 2) + 2024 is 2076.
The result of (26 * 2) + 2024 is 2076.
